# Subclassing `ASFProduct`

`ASFProduct` is the base class for all search result objects as of asf-search v7.0.0. There are several subclasses of `ASFProduct` that asf-search uses for specific platforms and product types with unique properties/functionality.

In [ ]:
import asf_search as asf
products = ['S1A_IW_SLC__1SDV_20231226T162948_20231226T163016_051828_0642C6_272F-SLC', 'S1_185682_IW2_20210224T161634_VV_035E-BURST','S1-GUNW-D-R-087-tops-20190301_20190223-161540-20645N_18637N-PP-7a85-v2_0_1-unwrappedPhase','ALPSRP111041130-RTC_HI_RES']
results = asf.product_search(product_list=products)
results

Notice the different type in the `results` list: `S1Product`, `S1BURSTProduct`, `ARIAS1GUNWProduct`, and `ALOSProduct`.
Each of these are subclasses of type `ASFProduct`.

Let's compare the `properties` of `S1Product` and `ALOSProduct`

In [ ]:
s1, s1Burst, ariaGunw, alos = results

def compare_properties(lhs: asf.ASFProduct, rhs: asf.ASFProduct):
    # Compares properties of two ASFProduct objects in a color coded table
    # values printed in red are missing from that product type altogether
    
    # Color Coding
    RED = '\033[31m'
    GREEN = '\033[32m'
    BLUE  = '\033[34m'
    RESET = '\033[0m'

    print(f'\t{GREEN}{type(lhs)}{RESET}\t{BLUE}{type(rhs)}{RESET}')
    
    keys = {*lhs.properties.keys(), *rhs.properties.keys()}
    for key in keys:
        print(f"{key}:\n\t{GREEN}{lhs.properties.get(key, f'{RED}None')}{RESET}\t{BLUE}{rhs.properties.get(key, f'{RED}None')}{RESET}\n")

compare_properties(s1, alos)

Notice a few properties (marked in red) are missing from each product properties dict. For example, `S1Product` has `pgeVersion`, while `ALOSProduct` has `offNadirAngle`, `faradayRotation`, and `insarStackId`. Moreover, their `baseline` field differs.

In [ ]:
print(f"{s1.properties['fileID']}\n\t{s1.baseline}\n")
print(f"{alos.properties['fileID']}\n\t{alos.baseline}")

`ASFProduct` has a class enum, `BaselineCalcType` that determines how asf-search will handle perpendicular stack calculations. Each subclass keeps track of their baseline calculation type via `ASFProduct.baseline_type`

The three `BaselineCalcType` types:
- `NONE` Cannot be used in baseline calculations
- `PRE_CALCULATED` Has pre-calculated insarBaseline value that will be used for perpendicular calculations
- `CALCULATED` Uses position/velocity state vectors and ascending node time for perpendicular calculations

Any subclass object that changes `baseline_type` from the default of `BaselineCalcType.NONE` is elligble for building a baseline stacking with `ASFProduct.stack()` (see the 4-Baseline_Search.ipynb example notebook for more examples of baseline stacking).

In [ ]:
print(f"Baseline Calculation Types")
print(f"ASFProduct:\t {asf.ASFProduct.baseline_type}")
print(f"ALOSProduct:\t {alos.baseline_type}")
print(f"S1Product:\t {s1.baseline_type}")

`ASFProduct` subclasses even have their own stack search option methods. The `ASFProduct` implementation of `get_stack_opts()` returns `None`, but subclasses like `S1Product` and `ALOSProduct` have different approaches.

In [ ]:
print(f"S1Product:\n{s1.get_stack_opts()}\n")
print(f"ALOSProduct:\n{alos.get_stack_opts()}")

Sublcasses of type `ASFProduct` can just as easily be parent classes to other subclasses, like `S1Product`, which is the parent class to `S1BurstProduct` and `ARIAS1GUNWProduct`.

In [ ]:
print("S1BurstProduct:")
print(f"\tburst dict:\n\t{s1Burst.properties['burst']}")
print(f"\nS1BurstProduct.get_stack_opts(): {s1Burst.get_stack_opts()}\n\n")

print(f"ARIAS1GUNWProduct:")
print(f"\tperpendicularBaseline: {ariaGunw.properties['perpendicularBaseline']}")
print(f"\tOrbit: {ariaGunw.properties['orbit']}")

Because `ASFProduct` is built for subclassing, that means users can provide their own custom subclasses.

In [ ]:
import copy
from typing import Union
from asf_search import ASFSearchOptions, ASFSession
from asf_search.ASFSearchOptions import ASFSearchOptions
from typing import Dict

class MyCustomS1Subclass(asf.S1Product):
    def __init__(
                #default ASFProduct arguments
                self, args: Dict = {}, session: ASFSession = ASFSession(), 
                #custom properties
                custom_properties: Dict = {}
            ):
        super().__init__(args, session)

        # unique property of MyCustomClass
        self.custom_properties = custom_properties
    
    # write custom methods
    def as_umm_json(self) -> Dict:
        return { 'umm': self.umm, 'meta': self.meta }
    
    # Override built in ASFProduct methods, like `geojson()`, `get_stack_opts()`, or `get_default_baseline_product_type()`
    def geojson(self) -> Dict:
        output = {
            **super().geojson()
        }

        output['properties'] = {
            **output['properties'],
            'customProperties': self.custom_properties
        }

        return output
    
    def get_stack_opts(self, opts: ASFSearchOptions = None) -> ASFSearchOptions:
        # use S1Product's stack opt already written functionality
        opts = super().get_stack_opts(opts)

        # but use some new product type to build the stack instead of S1Product's default, "SLC"
        opts.processingLevel = self.get_default_baseline_product_type()

        return opts
    
    @staticmethod
    def get_default_baseline_product_type() -> Union[str, None]:
        """
        Returns the product type to search for when building a baseline stack.
        """
        return 'NEW_PRODUCT_TYPE'

customS1SubclassProduct = MyCustomS1Subclass({'umm': s1.umm, 'meta': s1.meta}, session=s1.session, custom_properties={'customProperty': 'This is a special property'})

customS1SubclassProduct.geojson()

Notice the `customProperties` field in the output from `geojson()`.
`get_stack_opts()` should also use S1Product's stack ops, but with `NEW_PRODUCT_TYPE` instead of `SLC`:

In [ ]:
print(customS1SubclassProduct.get_stack_opts())